# Data Augmentation For Object Detection

This notebook serves as general manual to using this codebase. We cover all the major augmentations, as well as ways to combine them. 

In [81]:
from data_aug.data_aug import *
from data_aug.bbox_util import *
import numpy as np 
import cv2 
import matplotlib.pyplot as plt 
import pickle as pkl
import random
from skimage import io
import pandas as pd
from tqdm import tqdm
from ast import literal_eval
import os
%matplotlib inline

Estos códigos se corren y utilizan como lo indica el autor original, el repositorio para instalarse se puede consultar en:

https://github.com/Paperspace/DataAugmentationForObjectDetection

## Storage Format 

First things first, we define how the storage formats required for images to work. 
1. **The Image**: A OpenCV numpy array, of shape *(H x W x C)*. 
2. **Annotations**: A numpy array of shape *N x 5* where *N* is the number of objects, one represented by each row. 5 columns represent the top-left x-coordinate, top-left y-coordinate, bottom-right x-coordinate, bottom-right y-coordinate, and the class of the object. 

In [ ]:
img = cv2.imread("messi.jpg")[:,:,::-1]   #opencv loads images in bgr. the [:,:,::-1] does bgr -> rgb
bboxes = np.array([[x1,y1,x2,y2,c],...,[x1,y1,x2,y2,c]])

You can use the function `draw_rect` to plot the bounding boxes on an image. 

In [ ]:
plotted_img = draw_rect(img, bboxes)
plt.imshow(plotted_img)
plt.show()

Now, we can get started with our image augmentations. The first one is **Horizontal Flipping**. The function takes one arguement, *p* which is the probability that the image will be flipped. 

**Scaling**. Scales the image. If the argument *diff* is True, then the image is scaled with different values in the vertical and the horizontal directions, i.e. aspect ratio is not maintained. 

If the first argument is a float, then the scaling factors for both x and y directions are randomly sampled from *(- arg, arg)*. Otherwise, you can specify a tuple for this range.

**Translation**. Translates the image. If the argument *diff* is True, then the image is translated with different values in the vertical and the horizontal directions.

If the first argument is a float, then the translating factors for both x and y directions are randomly sampled from *(- arg, arg)*. Otherwise, you can specify a tuple for this range.

**Rotation**. Rotates the image. 

If the first argument is a int, then the rotating angle, in degrees, is sampled from *(- arg, arg)*. Otherwise, you can specify a tuple for this range.

**Shearing**. Sheares the image horizontally

If the first argument is a float, then the shearing factor is sampled from *(- arg, arg)*. Otherwise, you can specify a tuple for this range.

**Resizing**.  Resizes the image to square dimensions while keeping the aspect ratio constant.

The argument to this augmentation is the side of the square.

<h3>Aumento de datos para SIIM</h3>

In [151]:
#transformaciones de formato
def xywh_yolo(bb,size):
    cx=bb[0]+(bb[2]/2)
    cy=bb[1]+(bb[3]/2)
    xc=cx/size
    yc=cy/size
    wc=bb[2]/size
    hc=bb[3]/size
    return [xc,yc,wc,hc]

def yolon_to_xywh(bb,size):
    x=bb[0]*size-bb[2]*size/2
    y=bb[1]*size-bb[3]*size/2
    wp=bb[2]*size
    hp=bb[3]*size
    return [x, y, wp, hp]

def xywh_xiyixfyf(bbox):
    x1=bbox[0]
    y1=bbox[1]
    x2=bbox[0]+bbox[2]
    y2=bbox[1]+bbox[3]
    return [x1,y1,x2,y2]

def xiyixfyf_xywh(bbox):
    x = bbox[0]
    y = bbox[1]
    w = abs(bbox[2] - bbox[0])
    h = abs(bbox[3] - bbox[1])
    return [x, y, w, h]

def txt_bboxes(ruta):
    bboxes = []
    with open(ruta, 'r') as archivo:
        for linea in archivo:
            numeros = linea.strip().split()  # Dividir la línea en números
            bboxes.append([float(num) for num in numeros])  # Convertir números a enteros y añadir a la lista de listas
    return bboxes


#aplica transformaciones y guarda imagenes y anotaciones

#aplicando una escala
def escale_img(img,bboxes_format,gene):
    scale= random.uniform(0.2, 0.4)
    img_, bboxes_ = RandomScale(scale, diff = True)(img.copy(), bboxes_format.copy())
    
    if gene=='yolo': 
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            bbf=xiyixfyf_xywh(bb)
            bby=xywh_yolo(bbf,640)
            with open(root_dest_img.replace('images','labels')+name_img.replace('.png','_scaled.txt'), 'a') as file:
                file.write(str(ann)+' '+str(bby[0])+' '+str(bby[1])+' '+str(bby[2])+' '+str(bby[3])+'\n')
        
    else:
        xs=[]
        ys=[]
        xfs=[]
        yfs=[]
        c=[]
        n=[]
        io.imsave(root_dest_img+name_img.replace('.png','_scaled.png'), img_)
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            xs.append(int(bb[0]))
            ys.append(int(bb[1]))
            xfs.append(int(bb[2]))
            yfs.append(int(bb[3]))
            c.append(ann)
            n.append(root_dest_img+name_img.replace('.png','_scaled.png'))
        
        return xs,ys,xfs,yfs,c,n
            
    
    
        

#aplicando traslación
def translate_img(img,bboxes_format,gene):
    translate= random.uniform(0.1, 0.3)
    img_, bboxes_ = RandomTranslate(translate, diff = True)(img.copy(), bboxes_format.copy())
    
    if gene=='yolo': 
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            bbf=xiyixfyf_xywh(bb)
            bby=xywh_yolo(bbf,640)
            with open(root_dest_img.replace('images','labels')+name_img.replace('.png','_translated.txt'), 'a') as file:
                file.write(str(ann)+' '+str(bby[0])+' '+str(bby[1])+' '+str(bby[2])+' '+str(bby[3])+'\n')
    else:
        xs=[]
        ys=[]
        xfs=[]
        yfs=[]
        c=[]
        n=[]
        io.imsave(root_dest_img+name_img.replace('.png','_translated.png'), img_)
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            xs.append(int(bb[0]))
            ys.append(int(bb[1]))
            xfs.append(int(bb[2]))
            yfs.append(int(bb[3]))
            c.append(ann)
            n.append(root_dest_img+name_img.replace('.png','_translated.png'))
        
        return xs,ys,xfs,yfs,c,n
    
   

#aplicando rotación
def rotate_img(img,bboxes_format,gene,df):
    rotate = random.randint(10, 30)
    img_, bboxes_ = RandomRotate(rotate)(img.copy(), bboxes_format.copy())
    if gene=='yolo': 
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            bbf=xiyixfyf_xywh(bb)
            bby=xywh_yolo(bbf,640)
            with open(root_dest_img.replace('images','labels')+name_img.replace('.png','_rotated.txt'), 'a') as file:
                file.write(str(ann)+' '+str(bby[0])+' '+str(bby[1])+' '+str(bby[2])+' '+str(bby[3])+'\n')
    else:
        xs=[]
        ys=[]
        xfs=[]
        yfs=[]
        c=[]
        n=[]
        io.imsave(root_dest_img+name_img.replace('.png','_rotated.png'), img_)
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            xs.append(int(bb[0]))
            ys.append(int(bb[1]))
            xfs.append(int(bb[2]))
            yfs.append(int(bb[3]))
            c.append(ann)
            n.append(root_dest_img+name_img.replace('.png','_rotated.png'))
        
        return xs,ys,xfs,yfs,c,n
    
    

#aplica estiramiento
def shear_img(img,bboxes_format,gene,df):
    img_, bboxes_ = RandomShear(0.2)(img.copy(), bboxes_format.copy())
    if gene=='yolo': 
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            bbf=xiyixfyf_xywh(bb)
            bby=xywh_yolo(bbf,640)
            with open(root_dest_img.replace('images','labels')+name_img.replace('.png','_sheared.txt'), 'a') as file:
                file.write(str(ann)+' '+str(bby[0])+' '+str(bby[1])+' '+str(bby[2])+' '+str(bby[3])+'\n')
                
    else:
        xs=[]
        ys=[]
        xfs=[]
        yfs=[]
        c=[]
        n=[]
        io.imsave(root_dest_img+name_img.replace('.png','_sheared.png'), img_)
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            xs.append(int(bb[0]))
            ys.append(int(bb[1]))
            xfs.append(int(bb[2]))
            yfs.append(int(bb[3]))
            c.append(ann)
            n.append(root_dest_img+name_img.replace('.png','_sheared.png'))
        
        return xs,ys,xfs,yfs,c,n
    

#aplica una deformación compuesta por composicion de tranformaciones
def deform_img(img,bboxes_format,gene,df):
    scale= random.uniform(0.2, 0.4)
    translate= random.uniform(0.1, 0.3)
    rotate = random.randint(10, 30)
    seq = Sequence([RandomScale(scale), RandomTranslate(translate), RandomRotate(rotate), RandomShear(0.2)])
    img_, bboxes_ = seq(img.copy(), bboxes_format.copy())
    
    if gene=='yolo': 
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            bbf=xiyixfyf_xywh(bb)
            bby=xywh_yolo(bbf,640)
            with open(root_dest_img.replace('images','labels')+name_img.replace('.png','_deformed.txt'), 'a') as file:
                file.write(str(ann)+' '+str(bby[0])+' '+str(bby[1])+' '+str(bby[2])+' '+str(bby[3])+'\n')
    else:
        xs=[]
        ys=[]
        xfs=[]
        yfs=[]
        c=[]
        n=[]
        io.imsave(root_dest_img+name_img.replace('.png','_deformed.png'), img_)
        for bbox in bboxes_.tolist():
            bb=bbox[:-1]
            ann=int(bbox[4])
            xs.append(int(bb[0]))
            ys.append(int(bb[1]))
            xfs.append(int(bb[2]))
            yfs.append(int(bb[3]))
            c.append(ann)
            n.append(root_dest_img+name_img.replace('.png','_deformed.png'))
        
        return xs,ys,xfs,yfs,c,n
    
    
    

In [154]:
#aumento de datos para las clases con baja representación
mode='validation'
root_img='/home/jair/COVID/siimcovid/datasets/yolo/images/'+mode+'/'
root_dest_img='/home/jair/COVID/siimcovid/datasets/retinanet/aumento/images/'+mode+'/'
csv=pd.read_csv('/home/jair/COVID/siimcovid/'+mode+'_corrected.csv')
anotation={'Typical Appearance':0,'Negative for Pneumonia':1,'Indeterminate Appearance':2,'Atypical Appearance':3}
anotationi={0:'Typical Appearance',1:'Negative for Pneumonia',2:'Indeterminate Appearance',3:'Atypical Appearance'}
gene='retina'

resultados = {
    'rutas': [],
    'x1': [],
    'y1': [],
    'x2': [],
    'y2': [],
    'label': []
}
for indice, fila in tqdm(csv.iterrows()):
    if fila['clase']!='tt': 
        name_img=fila['id']
        ruta_img=root_img+name_img
        ruta_txt=root_img.replace('images','labels')+name_img.replace('png','txt')
        img = cv2.imread(ruta_img)[:,:,::-1]
        bboxes_yolo=literal_eval(fila['bboxes_norm'])
        bboxes_format=[]
        c=anotation[fila['clase']]
        for bbox in bboxes_yolo:
            bbox_xyhw=yolon_to_xywh(bbox,640)
            bbox_xf=xywh_xiyixfyf(bbox_xyhw)
            bbox_xf.append(c)
            bboxes_format.append(bbox_xf)
        
        bboxes_format=np.array(bboxes_format)
        try:
            xs,ys,xfs,yfs,c,n=escale_img(img,bboxes_format,gene)
            for i in range(len(n)):
                resultados['x1'].append(xs[i])
                resultados['y1'].append(ys[i])
                resultados['x2'].append(xfs[i])
                resultados['y2'].append(yfs[i])
                resultados['label'].append(anotationi[c[i]])
                resultados['rutas'].append(n[i])
        except Exception as e:
            pass
        try:
            xs,ys,xfs,yfs,c,n=translate_img(img,bboxes_format,gene)
            for i in range(len(n)):
                resultados['x1'].append(xs[i])
                resultados['y1'].append(ys[i])
                resultados['x2'].append(xfs[i])
                resultados['y2'].append(yfs[i])
                resultados['label'].append(anotationi[c[i]])
                resultados['rutas'].append(n[i])
        except Exception as e:
            pass
        try:
            xs,ys,xfs,yfs,c,n=rotate_img(img,bboxes_format,gene)
            for i in range(len(n)):
                resultados['x1'].append(xs[i])
                resultados['y1'].append(ys[i])
                resultados['x2'].append(xfs[i])
                resultados['y2'].append(yfs[i])
                resultados['label'].append(anotationi[c[i]])
                resultados['rutas'].append(n[i])
        except Exception as e:
            pass
        try:
            xs,ys,xfs,yfs,c,n=shear_img(img,bboxes_format,gene)
            for i in range(len(n)):
                resultados['x1'].append(xs[i])
                resultados['y1'].append(ys[i])
                resultados['x2'].append(xfs[i])
                resultados['y2'].append(yfs[i])
                resultados['label'].append(anotationi[c[i]])
                resultados['rutas'].append(n[i])

        except Exception as e:
            pass
        try:
            xs,ys,xfs,yfs,c,n=deform_img(img,bboxes_format,gene)
            for i in range(len(n)):
                resultados['x1'].append(xs[i])
                resultados['y1'].append(ys[i])
                resultados['x2'].append(xfs[i])
                resultados['y2'].append(yfs[i])
                resultados['label'].append(anotationi[c[i]])
                resultados['rutas'].append(n[i])
        except Exception as e:
            pass

df= pd.DataFrame(resultados)

1155it [03:40,  5.24it/s]


In [155]:
df.to_csv('/home/jair/COVID/siimcovid/datasets/retinanet/aumento/images/'+mode+'_transf.csv', index=False)

In [149]:
mode='validation'
root_img='/home/jair/COVID/siimcovid/datasets/yolo/images/'+mode+'/'
root_dest_img='/home/jair/COVID/siimcovid/datasets/retinanet/aumento/images/'+mode+'/'
csv=pd.read_csv('/home/jair/COVID/siimcovid/'+mode+'_corrected.csv')
anotation={'Typical Appearance':0,'Negative for Pneumonia':1,'Indeterminate Appearance':2,'Atypical Appearance':3}
anotationi={0:'Typical Appearance',1:'Negative for Pneumonia',2:'Indeterminate Appearance',3:'Atypical Appearance'}
resultados = {
    'rutas': [],
    'x1': [],
    'y1': [],
    'x2': [],
    'y2': [],
    'label': []
}
for indice, fila in tqdm(csv.iterrows()):
    if fila['clase']!='tt': 
        name_img=fila['id']
        bboxes_yolo=literal_eval(fila['bboxes_norm'])
        for bbox in bboxes_yolo:
            bbox_xyhw=yolon_to_xywh(bbox,640)
            bbox_xf=xywh_xiyixfyf(bbox_xyhw)
            resultados['x1'].append(int(bbox_xf[0]))
            resultados['y1'].append(int(bbox_xf[1]))
            resultados['x2'].append(int(bbox_xf[2]))
            resultados['y2'].append(int(bbox_xf[3]))
            resultados['label'].append(fila['clase'])
            resultados['rutas'].append(root_dest_img+name_img)

df= pd.DataFrame(resultados)

1155it [00:00, 5292.08it/s]


In [150]:
df.to_csv('/home/jair/COVID/siimcovid/datasets/retinanet/aumento/images/'+mode+'_orig.csv', index=False)

In [9]:
#borrando imagenes sin anotaciones
mode='validation'
root='/home/jair/COVID/siimcovid/datasets/retinanet/aumento/images/'
archivos=os.listdir(root+mode)

i=0
for archivo in archivos:
    if not os.path.exists(root.replace('images','labels')+mode+'/'+archivo.replace('png','txt')):
        os.remove(root+mode+'/'+archivo)



In [158]:
mode='validation'
csv=pd.read_csv('/home/jair/COVID/siimcovid/datasets/retinanet/aumento/images/'+mode+'_transf.csv')


In [10]:
#conteos por clases sin aumento
info=pd.read_csv('/home/jair/COVID/siimcovid/info.csv')
conteo = info['label'].value_counts()
conteo

label
Typical Appearance          2738
Negative for Pneumonia      1665
Indeterminate Appearance    1000
Atypical Appearance          375
Name: count, dtype: int64

In [41]:
#contando las imagenes que se agregaron para indeterminados y atipicos
root_search='/home/jair/COVID/siimcovid/datasets/retinanet/aumento/images/train/'
ind=0
ati=0
tip=0
neg=0
for indice, fila in tqdm(info.iterrows()):
    name=fila['rutas'].split('/')[-1].replace('dcm','png')

    if fila['label']=='Indeterminate Appearance':
        if os.path.exists(root_search+name):
            if os.path.exists(root_search+name.replace('.png','_scaled.png')):
                ind=ind+1
            if os.path.exists(root_search+name.replace('.png','_translated.png')):
                ind=ind+1
            if os.path.exists(root_search+name.replace('.png','_rotated.png')):
                ind=ind+1
            if os.path.exists(root_search+name.replace('.png','_sheared.png')):
                ind=ind+1
            if os.path.exists(root_search+name.replace('.png','_deformed.png')):
                ind=ind+1
        elif os.path.exists(root_search.replace('train','validation')+name):
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_scaled.png')):
                ind=ind+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_translated.png')):
                ind=ind+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_rotated.png')):
                ind=ind+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_sheared.png')):
                ind=ind+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_deformed.png')):
                ind=ind+1
    if fila['label']=='Atypical Appearance':
        if os.path.exists(root_search+name):
            if os.path.exists(root_search+name.replace('.png','_scaled.png')):
                ati=ati+1
            if os.path.exists(root_search+name.replace('.png','_translated.png')):
                ati=ati+1
            if os.path.exists(root_search+name.replace('.png','_rotated.png')):
                ati=ati+1
            if os.path.exists(root_search+name.replace('.png','_sheared.png')):
                ati=ati+1
            if os.path.exists(root_search+name.replace('.png','_deformed.png')):
                ati=ati+1
        elif os.path.exists(root_search.replace('train','validation')+name):
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_scaled.png')):
                ati=ati+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_translated.png')):
                ati=ati+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_rotated.png')):
                ati=ati+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_sheared.png')):
                ati=ati+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_deformed.png')):
                ati=ati+1
    if fila['label']=='Typical Appearance':
        if os.path.exists(root_search+name):
            if os.path.exists(root_search+name.replace('.png','_scaled.png')):
                tip=tip+1
            if os.path.exists(root_search+name.replace('.png','_translated.png')):
                tip=tip+1
            if os.path.exists(root_search+name.replace('.png','_rotated.png')):
                tip=tip+1
            if os.path.exists(root_search+name.replace('.png','_sheared.png')):
                tip=tip+1
            if os.path.exists(root_search+name.replace('.png','_deformed.png')):
                tip=tip+1
        elif os.path.exists(root_search.replace('train','validation')+name):
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_scaled.png')):
                tip=tip+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_translated.png')):
                tip=tip+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_rotated.png')):
                tip=tip+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_sheared.png')):
                tip=tip+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_deformed.png')):
                tip=tip+1
    if fila['label']=='Negative for Pneumonia':
        if os.path.exists(root_search+name):
            if os.path.exists(root_search+name.replace('.png','_scaled.png')):
                neg=neg+1
            if os.path.exists(root_search+name.replace('.png','_translated.png')):
                neg=neg+1
            if os.path.exists(root_search+name.replace('.png','_rotated.png')):
                neg=neg+1
            if os.path.exists(root_search+name.replace('.png','_sheared.png')):
                neg=neg+1
            if os.path.exists(root_search+name.replace('.png','_deformed.png')):
                neg=neg+1
        elif os.path.exists(root_search.replace('train','validation')+name):
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_scaled.png')):
                neg=neg+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_translated.png')):
                neg=neg+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_rotated.png')):
                neg=neg+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_sheared.png')):
                neg=neg+1
            if os.path.exists(root_search.replace('train','validation')+name.replace('.png','_deformed.png')):
                neg=neg+1

5778it [00:00, 9381.68it/s] 


In [37]:
print('Atipicos agregados',ati)
print('Indeterminados agregados',ind)
print('nega add',neg)
print('tipi add',tip)

Atipicos agregados 0
Indeterminados agregados 0
nega add 0
tipi add 0


Conteos despues de aumento

Typical Appearance          2738

Negative for Pneumonia      1665

Indeterminate Appearance    5462

Atypical Appearance         2050

In [78]:
mode='validation'
root='/home/jair/COVID/siimcovid/datasets/yolo/labels/'
names={0:'Typical Appearance',1:'Negative for Pneumonia',2: 'Indeterminate Appearance',3: 'Atypical Appearance'}
archivos=os.listdir(root+mode)
ids=[]
bboxes=[]
clase=[]
df = pd.DataFrame(columns=['id','bboxes_norm','clase'])
for archivo in archivos:
    idn=archivo.replace('txt','png')
    Cbboxes=txt_bboxes(root+mode+'/'+archivo)
    clss=int(Cbboxes[0][0])
    ids.append(idn)
    clase.append(names[clss])
    bbs=[]
    for bb in Cbboxes:
        bbs.append(bb[1:])
    bboxes.append(str(bbs))          

In [79]:
df['id']=ids
df['bboxes_norm']=bboxes
df['clase']=clase

In [80]:
df.to_csv('/home/jair/COVID/siimcovid/'+mode+'_corrected.csv', index=False)